# Generate QnA synthetic dataset from CSV

This is another common case. If image url information is included, change this url to a summary result for that image.

In [2]:
from dotenv import load_dotenv
import os, shutil, random
from unstructured.cleaners.core import clean_bullets, clean_extra_whitespace, remove_punctuation
from langchain_community.document_loaders import UnstructuredFileLoader, UnstructuredMarkdownLoader, UnstructuredAPIFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader, UnstructuredCSVLoader
from util.preprocess import convert_html_to_md

load_dotenv()
raw_data_dir = "raw_data"
file_path = f"{raw_data_dir}/prod-st-excel/pivot_spec_mx_smartphone_s24.csv"


## 1. Read & Preprocess CSV file
---

In [3]:
%%time

loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    },    
)
docs = loader.load()

CPU times: user 5.65 ms, sys: 1.87 ms, total: 7.52 ms
Wall time: 21.6 ms


In [4]:
preprocessed_docs = []
for doc in docs:
    md = convert_html_to_md(doc.page_content)
    preprocessed_docs.append(md)

print(preprocessed_docs[0])

모델코드: SM-S921NLBEKOO
운영체제: Android
센서: 가속도 센서,기압 센서,지문 센서,자이로 센서,지자기 센서,홀 센서,조도 센서,근접 센서
S펜 지원: 없음
디스플레이\_크기 (Main Display): 156.4 mm
디스플레이\_해상도 (Main): 2340 x 1080 (FHD+)
디스플레이\_종류 (Main): Dynamic AMOLED 2X
디스플레이\_색심도 (Main): 16 M
프로세서\_CPU 속도: 3.2 GHz,2.9 GHz,2.6 GHz,1.95 GHz
프로세서\_CPU 종류: Deca-Core
카메라\_후면 카메라 - 화소 (Multiple): 50.0 MP + 10.0 MP + 12.0 MP
카메라\_후면 카메라 - 조리개 값 (Multiple): F1.8 , F2.4 , F2.2
카메라\_후면 카메라 - 오토 포커스: 예
카메라\_후면 카메라 - OIS: 예
카메라\_후면 카메라 줌: 없음
카메라\_전면 카메라 - 화소: 12.0 MP
카메라\_전면 카메라 - 조리개 값: F2.2
카메라\_전면 카메라 - 오토 포커스: 예
카메라\_전면 카메라 - OIS: 없음
카메라\_후면 카메라 - 플래쉬: 예
카메라\_전면 카메라 - 플래쉬: 없음
카메라\_동영상 녹화 해상도: UHD 8K (7680 x 4320) @30fps
메모리/스토리지(저장 용량)\_메모리 (GB): 8 GB
메모리/스토리지(저장 용량)\_스토리지(저장 용량) (GB): 256 GB
메모리/스토리지(저장 용량)\_사용 가능한 스토리지(저장 용량) (GB): 232.7 GB
메모리/스토리지(저장 용량)\_외장 스토리지(저장 용량) 지원: 없음
네트워크\_SIM 개수: Dual-SIM
네트워크\_SIM 사이즈: 없음
네트워크\_SIM 슬롯 타입: SIM 1 + eSIM / Dual eSIM
네트워크 (S/W 사용)\_2G GSM: GSM850,GSM900,DCS1800,PCS1900
네트워크 (S/W 사용)\_3G UMTS: B1(2100),B2(1900

## 2. Construct QnA Pairs
---

In [5]:
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from util.qa_pair import get_qna_prompt_template, QAPair

llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=1024,
    openai_api_version="2024-05-01-preview",
    azure_deployment="gpt-4o"                       
)

parser = JsonOutputParser(pydantic_object=QAPair)
prompt = get_qna_prompt_template()
chain = prompt | llm | parser

Prepare input batch

In [6]:
input_batch = []

for doc in preprocessed_docs:
    dic = {"context": doc, "domain": "Samsung Galaxy S series Smartphone", "num_questions": "1"}
    input_batch.append(dic)

In [9]:
%%time
qa_pair = chain.batch(input_batch, {"max_concurrency": 5})

In [10]:
# Remove duplicate items
qa_pair = [dict(t) for t in {tuple(d.items()) for d in qa_pair}]


## 3. Save to jsonl for fine-tuning
---

In [ ]:
import json
from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = "You are an AI assistant that guides product specifications for the Galaxy series."
#system_prompt_msg = "You're an AI assistant that guides a user to general information of Galaxy Series."
save_filename = "cs-prod-spec-s24"
oai_qa_pair = convert_to_oai_format(qa_pair)

save_jsonl(qa_pair, f"{output_dir}/{save_filename}.jsonl")
save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")